In [1]:
import lightcones.linalg as ll
from lightcones.linalg import kron
from lightcones.models import spinfull_fermions


U = 1
t = 0.05

num_iterations = 100

m = 100
l = 3 + 1

# iteration 0: preparatory

# left subsystem

f = spinfull_fermions(l)

In [2]:
H_hopp = sum([sum([- t * (f.a_dag[s][i] @ f.a[s][i - 1] + f.a_dag[s][i - 1] @ f.a[s][i]) for s in range(2)]) for i in range(l)])
H_S = H_hopp + sum([U * f.n[0][i] @ f.n[1][i] for i in range(l)])

In [3]:
H_S_ = kron(H_S, f.eye)

In [4]:
import scipy

In [5]:

H_S_2 = scipy.sparse.kron(H_S, f.eye, format = 'csc')

In [6]:
H_S_2

<Compressed Sparse Column sparse matrix of dtype 'complex128'
	with 306944 stored elements and shape (65536, 65536)>

In [7]:
a_S = f.a[:][-1]
a_S_dag = f.a_dag[:][-1]

In [9]:
a_S

[<Compressed Sparse Column sparse matrix of dtype 'complex128'
 	with 2048 stored elements and shape (4096, 4096)>,
 <Compressed Sparse Column sparse matrix of dtype 'complex128'
 	with 2048 stored elements and shape (4096, 4096)>,
 <Compressed Sparse Column sparse matrix of dtype 'complex128'
 	with 2048 stored elements and shape (4096, 4096)>,
 <Compressed Sparse Column sparse matrix of dtype 'complex128'
 	with 2048 stored elements and shape (4096, 4096)>,
 <Compressed Sparse Column sparse matrix of dtype 'complex128'
 	with 2048 stored elements and shape (4096, 4096)>,
 <Compressed Sparse Column sparse matrix of dtype 'complex128'
 	with 2048 stored elements and shape (4096, 4096)>]

In [8]:
a_S_ = kron(a_S, f.eye)

In [10]:
a_S_dag_ = kron(a_S_dag, f.eye)

In [11]:
H_E_ = kron(f.eye, H_S)

In [29]:
import numpy as np
import scipy
import lightcones.linalg as ll
from lightcones.linalg import kron
from lightcones.models import spinfull_fermions
import lightcones.space as sp
import math


U = 1
t = 0.05

num_iterations = 10

m = 10
l = 3 + 1

# iteration 0: preparatory

# left subsystem

f = spinfull_fermions(l)

H_hopp = sum([sum([- t * (f.a_dag[s][i] @ f.a[s][i - 1] + f.a_dag[s][i - 1] @ f.a[s][i]) for s in range(2)]) for i in range(l)])
H_S = H_hopp + sum([U * f.n[0][i] @ f.n[1][i] for i in range(l)])

H_S_ = kron(H_S, f.eye)

a_S = f.a[:][-1]
a_S_dag = f.a_dag[:][-1]

a_S_ = kron(a_S, f.eye)
a_S_dag_ = kron(a_S_dag, f.eye)

# right subsystem

# fermion Hamiltonian is always even
H_E_ = kron(f.eye, H_S)

# a and a_dag are odd operators
a_E_ = kron(f.parity, a_S)
a_E_dag_ = kron(f.parity, a_S_dag)

# joint system Hamiltonian
H_SE = H_S_ + H_E_ - sum([t * (a_E_dag_[s] @ a_S_[s] + a_S_dag_[s] @ a_E_[s]) for s in range(2)])

# initial lanczos wavefunction for
# half filled case:
psi0 = np.zeros(f.dimension**2, dtype = complex)
psi0[0] = 1.0
for i in range(l):
    # populate left part
    psi0 = (kron(f.a_dag[0][i], f.eye) + kron(f.a_dag[1][i], f.eye)) @ psi0 / math.sqrt(2.0)
    # popular right part
    psi0 = (kron(f.parity, f.a_dag[0][i]) + kron(f.parity, f.a_dag[1][i])) @ psi0 / math.sqrt(2.0)

print("psi0 norm:")
print(np.vdot(psi0, psi0))

# total occupation of left subsystem:
N_left = sum([sum([f.a_dag[s][i] @ f.a[s][i] for s in range(2)]) for i in range(l)])
N_left_ = kron(N_left, f.eye)

print("psi0 left part occupation:")
print(np.vdot(psi0, N_left_ @ psi0))

# now start regular iterations

for i in range(num_iterations):

    # find ground state
    E_GND, Psi_GND = ll.lancz_gnd_state(psi0, H_SE, 10)
    
    print("E_GND = ", E_GND)
    
    print("Psi_GND norm:")
    print(np.vdot(Psi_GND, Psi_GND))
    
    # partial trace
    bp = sp.bipartite(f.dimension, f.dimension)
    rho = bp.trace_out_R(Psi_GND)
    
    # find eigenvalues sorted in descending order
    pi, T = ll.find_eigs_descending(rho)
    
    print("probability norm:")
    print(sum(pi))
    
    print("Schmidt components:")
    print(pi[:m])
    
    # keep only m largest eigenstates
    # columns are eigenstates
    T = scipy.sparse.csc_matrix(T[:, : m])

    # switch to the frame of revelant eigenstates
    T_dag = scipy.sparse.csc_matrix(T.conj().T)
    
    N_left_T = T_dag @ N_left @ T
    
    print("Population of Schmidt components:")
    left_N = N_left_T.diagonal().real
    print(left_N)
    
    assert math.isclose(left_N[0], l, rel_tol=1e-9, abs_tol=1e-9), \
        "The most important Schmidt component is not at half filling."
    
    H_T = T_dag @ H_S @ T

    # boundary operators also get renormalized
    a_T = [T_dag @ a_S[s] @ T for s in range(2)]
    a_T_dag = [T_dag @ a_S_dag[s] @ T for s in range(2)]

    # new parity operator
    parity_T = T_dag @ f.parity @ T
    eye_T = ll.eye(m)
    
    # add new site
    g = spinfull_fermions(1)
    
    # new boundary operators
    
    # a and a_dag are odd operators
    a_S = kron(parity_T, [g.a[s][0] for s in range(2)])
    a_S_dag = kron(parity_T, [g.a_dag[s][0] for s in range(2)])
    
    a_T_ = kron(a_T, g.eye)
    a_T_dag_ = kron(a_T_dag, g.eye)
    
    # n is even operator
    n_ = kron(eye_T, [g.n[s][0] for s in range(2)])
    
    # 
    H_T_ = kron(H_T, g.eye)
    
    #
    H_S = H_T_ - t * sum([a_S_dag[s] @ a_T_[s] + a_T_dag_[s] @ a_S[s] for s in range(2)]) + U * n_[0] @ n_[1]
    
    eye_T_ = kron(eye_T, g.eye)
    
    # left subsystem
    
    H_S_ = kron(H_S, eye_T_)
    
    a_S_ = kron(a_S, eye_T_)
    a_S_dag_ = kron(a_S_dag, eye_T_)

    # right subsystem

    parity_T_ = kron(parity_T, g.parity)
    
    # fermion Hamiltonian is always even
    H_E_ = kron(eye_T_, H_S)

    # a and a_dag are odd operators
    a_E_ = kron(parity_T_, a_S)
    a_E_dag_ = kron(parity_T_, a_S_dag)
    
    # joint system Hamiltonian

    H_SE = H_S_ + H_E_ - sum([t * (a_E_dag_[s] @ a_S_[s] + a_S_dag_[s] @ a_E_[s]) for s in range(2)])
    
    # update Lanczos initial condition
    # Take the most important Schmidt component |phi_0>
    # and let |psi0> =|phi0>|0>|0>|phi0> 
    
    

psi0 norm:
(0.9999999999999983+0j)
psi0 left part occupation:
(3.9999999999999933+0j)
E_GND =  [-0.03465369]
Psi_GND norm:
(1.0000000000000033+0j)
probability norm:
1.000000000000008
Schmidt components:
[9.02161892e-01 6.23253776e-02 1.94681965e-02 1.00040158e-02
 1.83148190e-03 1.38405770e-03 5.95134507e-04 5.95134507e-04
 5.95134507e-04 5.95134507e-04]
Population of Schmidt components:
[4. 4. 4. 4. 4. 4. 5. 5. 3. 3.]


ValueError: dimension mismatch

In [9]:
T

<Compressed Sparse Column sparse matrix of dtype 'complex128'
	with 21539 stored elements and shape (256, 100)>

In [4]:
H_S

<Compressed Sparse Column sparse matrix of dtype 'complex128'
	with 116797 stored elements and shape (400, 400)>

In [5]:
H_S.eliminate_zeros()

In [6]:
H_S

<Compressed Sparse Column sparse matrix of dtype 'complex128'
	with 116797 stored elements and shape (400, 400)>

In [3]:
eye_T_

<Compressed Sparse Column sparse matrix of dtype 'float64'
	with 400 stored elements and shape (400, 400)>

In [8]:
parity_T_

<Compressed Sparse Column sparse matrix of dtype 'complex128'
	with 39208 stored elements and shape (400, 400)>

In [7]:
parity_T_.eliminate_zeros()

In [7]:
a_S = kron(parity_T, [g.a[s][0] for s in range(2)])

Exception: Unsupported types for kron

In [10]:
T

<Compressed Sparse Column sparse matrix of dtype 'complex128'
	with 21539 stored elements and shape (256, 100)>

In [11]:
T_dag

<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 21539 stored elements and shape (100, 256)>

In [9]:
f.parity

<Compressed Sparse Column sparse matrix of dtype 'float64'
	with 256 stored elements and shape (256, 256)>

In [8]:
parity_T

<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 9802 stored elements and shape (100, 100)>

In [3]:
a_T_ = kron(a_T, g.eye)

Exception: Unsupported types for kron

In [4]:
g.eye

<Compressed Sparse Column sparse matrix of dtype 'float64'
	with 4 stored elements and shape (4, 4)>

In [5]:
a_T

[array([[-5.53213328e-18+0.j,  5.63835956e-18+0.j,  1.31505284e-17+0.j,
         ..., -2.48265053e-09+0.j, -4.33333432e-09+0.j,
         -1.45791600e-08+0.j],
        [ 5.13613778e-01+0.j,  6.25612326e-16+0.j,  1.58979287e-13+0.j,
         ...,  1.75940207e-06+0.j, -3.45751545e-06+0.j,
          8.36257890e-06+0.j],
        [-1.96601364e-03+0.j,  8.99932045e-17+0.j, -5.90317161e-16+0.j,
         ..., -2.36680590e-05+0.j, -1.82980949e-05+0.j,
         -8.55664372e-06+0.j],
        ...,
        [-2.87613855e-09+0.j, -7.72047188e-07+0.j, -8.78948754e-05+0.j,
         ..., -1.04299613e-02+0.j,  2.17782286e-02+0.j,
         -7.20550004e-02+0.j],
        [-5.66488830e-08+0.j, -2.16150237e-07+0.j, -4.58175553e-05+0.j,
         ..., -2.72613784e-02+0.j,  5.40979269e-02+0.j,
         -9.15065830e-02+0.j],
        [ 2.30361961e-08+0.j,  1.65705939e-07+0.j, -1.08476565e-05+0.j,
         ...,  1.58975803e-03+0.j, -1.92734273e-02+0.j,
          7.93118654e-03+0.j]]),
 array([[-1.06689230e-17+0.j,  

In [2]:
kron(parity_T, [g.a[s][0] for s in range(2)])

[<Compressed Sparse Column sparse matrix of dtype 'complex128'
 	with 19604 stored elements and shape (400, 400)>,
 <Compressed Sparse Column sparse matrix of dtype 'complex128'
 	with 19604 stored elements and shape (400, 400)>]

In [19]:
print(type(parity_T))

<class 'numpy.ndarray'>


In [25]:
import scipy
import numpy

In [26]:
def is_sparse_matrix(a):
    if isinstance(a, scipy.sparse.csc_matrix):
        return True
    if isinstance(a, numpy.ndarray) and a.ndim == 2:
        return True

In [27]:
is_sparse_matrix(parity_T)

True

In [17]:
parity_T

array([[ 1.00000000e+00+0.j,  1.02565360e-17+0.j, -5.15329956e-17+0.j,
        ...,  3.46944695e-18+0.j, -8.67361738e-19+0.j,
        -4.33680869e-18+0.j],
       [ 1.02565360e-17+0.j, -1.00000000e+00+0.j,  1.52330405e-17+0.j,
        ...,  5.26254371e-17+0.j,  9.46401767e-17+0.j,
        -1.24024952e-17+0.j],
       [-5.15329956e-17+0.j,  1.52330405e-17+0.j, -1.00000000e+00+0.j,
        ..., -5.46388830e-16+0.j, -4.40976505e-17+0.j,
         2.79783261e-17+0.j],
       ...,
       [ 3.46944695e-18+0.j,  5.26254371e-17+0.j, -5.46388830e-16+0.j,
        ..., -4.22686088e-01+0.j,  3.86737198e-01+0.j,
         4.28768176e-02+0.j],
       [-8.67361738e-19+0.j,  9.46401767e-17+0.j, -4.40976505e-17+0.j,
        ...,  3.86737198e-01+0.j, -2.14610628e-01+0.j,
         1.05354078e-01+0.j],
       [-4.33680869e-18+0.j, -1.24024952e-17+0.j,  2.79783261e-17+0.j,
        ...,  4.28768176e-02+0.j,  1.05354078e-01+0.j,
        -6.65462964e-01+0.j]])

In [18]:
[g.a[s][0] for s in range(2)]

[<Compressed Sparse Column sparse matrix of dtype 'complex128'
 	with 2 stored elements and shape (4, 4)>,
 <Compressed Sparse Column sparse matrix of dtype 'complex128'
 	with 2 stored elements and shape (4, 4)>]

In [2]:
import numpy as np

In [16]:
psi = np.zeros(f.fermions.states.dimension ** 4, dtype = complex)
psi[:] = 1.0 


In [11]:
f.fermions.states.dimension ** 4

65536

In [9]:
H_SE

<Compressed Sparse Column sparse matrix of dtype 'complex128'
	with 648799 stored elements and shape (65536, 65536)>

In [21]:
import math

In [23]:
for i in range(100):
    psi = H_SE @ psi
    psi = psi / math.sqrt(np.vdot(psi, psi).real)

In [7]:
from scipy.sparse.linalg import eigsh
E_GND, Psi_GND = eigsh(H_SE, k=1, which='SM')

KeyboardInterrupt: 

In [ ]:
import lightcones.linalg as la
from lightcones.linalg import kron

U = 1
t = 0.05

#eps_cut = 0.00001

num_iterations = 100

m = 100
l = 5 + 1

# iteration 0: preparatory

# left subsystem

f = spinfull_fermions(l)

H_hopp = sum([sum([- t * (f.a_dag[s][i] @ f.a[s][i - 1] + f.a_dag[s][i - 1] @ f.a[s][i]) for s in range(2)]) for i in range(l)])
H_S = H_hopp + sum([U * f.n[0][i] @ f.n[1][i] for i in range(l)])

H_S_ = kron(H, f.eye)

a_S = f.a[:][-1]
a_S_dag = f.a_dag[:][-1]

a_S_ = kron(a_S, f.eye)
a_S_dag_ = kron(a_S_dag, f.eye)

# right subsystem

# fermion Hamiltonian is always even
H_E_ = kron(f.eye, H_S)

# a and a_dag are odd operators
a_E_ = kron(f.parity, a_S)
a_E_dag_ = kron(f.parity, a_S_dag)

# joint system Hamiltonian

H_SE = H_S_ + H_E_ - sum([t * (a_E_dag_[s] @ a_S_[s] + a_S_dag_[s] @ a_E_[s]) for s in range(2)])

# now start regular iterations

for i in range(num_iterations):

    # find ground state
    
    from scipy.sparse.linalg import eigsh
    E_GND, Psi_GND = eigsh(H_SE, k=1, which='SM')

    # partial trace

    r = reduce_bipartite(f.states.dimension, f.state_dimension, trace_over = 'R')

    rho = r(Psi_GND)

    # find eigenvalues sorted in ascending order

    pi, T = la.find_eigs_descending(rho)

    # keep only m largest eigenstates
    # columns are eigenstates
    T = T[:, : m] 

    # switch to the frame of revelant eigenstates
    T_dag = T.conj().T
    H_T = T_dag @ H_S @ T

    a_T = [T_dag @ a_S[s] @ T for s in range(2)]
    a_T_dag = [T_dag @ a_S_dag[s] @ T for s in range(2)]

    # new parity operator
    parity_T = T_dag @ f.parity @ T
    eye_T = la.eye(m)
    
    # add new site
    
    g = spinfull_fermions(1)
    
    # a and a_dag are odd operators
    a_S = kron(parity_T, g.a)
    a_S_dag = kron(parity_T, g.a_dag)
    
    a_T_ = kron(a_T, g.eye)
    a_T_dag_ = kron(a_T_dag, g.eye)
    
    # n is even operator
    n_ = kron(eye_T, g.n[:, 0])
        
    H_S = H_T - t * sum([a_S_dag[s] @ a_T_[s] + a_T_dag_[s] @ a_S[s] for s in range(2)]) + U * n_[0] @ n_[1]
    
    eye_T_ = kron(eye_T, g.eye)
    
    # left subsystem
    
    H_S_ = kron(H_S, eye_T_)
    
    a_S_ = kron(a_S, eye_T_)
    a_S_dag_ = kron(a_S_dag, eye_T_)

    # right subsystem

    parity_T_ = kron(parity_T, g.parity)

    # fermion Hamiltonian is always even
    H_E_ = kron(eye_T_, H_S)

    # a and a_dag are odd operators
    a_E_ = kron(parity_T_, a_S)
    a_E_dag_ = kron(parity_T_, a_S_dag)
    
    # joint system Hamiltonian

    H_SE = H_S_ + H_E_ - sum([t * (a_E_dag_[s] @ a_S_[s] + a_S_dag_[s] @ a_E_[s]) for s in range(2)])


    
    
    
    







In [ ]:
f = spinfull_fermions(n_modes)

f.states.state_at()
f.states.index_of()

f.vac()

f.up.a[i]
f.up.a_dag[i]

f.down.a[i]
f.down.a_dag[i]

f.a[s,i] # s = 0 (up) 1 (down)
f.a_dag[s, i]

f.n[s, i]

f.parity

f.zero

f.eye

In [ ]:
import numpy as np

array = np.empty((3, 3), dtype=object)
array[0, 0] = Node(...)